# Import

In [3]:
import torch
from torch import FloatTensor, LongTensor
import math

In [5]:
from load_script_deep_framework import load_dataset

In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
import torch.nn as nn

In [8]:
?nn.Linear.forward

# Loading the dataset

In [9]:
train,target_train = load_dataset()

In [10]:
test,target_test = load_dataset()

# Generic Module Class

In [11]:
class Module():
    
    def forward(self,input):
        raise NotImplemented
        
    def backward(self,input):
        raise NotImplemented
            
    def param(self):
        return
    
    def __call__(self,*input):
        return self.forward(*input)
    
    def zero_grad_(self):
        return
    
    def update_weights_(self,epsilon):
        return

# Specific classes

In [12]:
class Linear(Module):
    
    def __init__(self,input_features,output_features):
        super(Linear,self).__init__()
        
        self._input_features=input_features
        self._output_features=output_features
    
        self._weights=torch.randn(self._output_features,self._input_features)*1e-1
        self._gradient=torch.zeros(self._weights.shape)
    
    def forward(self,input):
        self._input=input.view(-1)
    
        self._output=self._weights.mv(self._input)
        return self._output.clone()
        
    def backward(self,d_dy):
        self._gradient.add_(d_dy.view(-1,1)*self._input.view(1,-1))
        
        d_dx=self._weights.t().mv(d_dy)
        return d_dx
    
    def zero_grad_(self):
        self._gradient.zero_()
        
    def update_weights_(self,epsilon):
        self._weights.add_(-epsilon*self._gradient)
    

In [13]:
class ReLU(Module):
    def __init__(self):
        super(ReLU,self).__init__()
        
    def forward(self,input):
        self._input=input.clone()
        
        self._output=self._input.clone()
        self._output[self._output<0] = 0

        return self._output.clone()
    
    def backward(self,d_dy):
        
        d_dx = d_dy.clone()
        d_dx[self._input<0] = 0
        
        return d_dx
        

In [14]:
class Tanh(Module):
    def __init__(self):
        super(Tanh,self).__init__()
        
    def forward(self,input):
        self._input=input.clone()
        
        self._output=self._input.tanh()
        
        return self._output.clone()
    
    def backward(self,d_dy):
        d_dx= (1-self._input.tanh()**2)*d_dy
        
        return d_dx
        

In [15]:
class LossMSE(Module):
    def __init__(self):
        super(LossMSE,self).__init__()
        
    def forward(self,input,target):
        self._input = input - target
        self._output = (self._input).pow(2).sum()
        return self._output
        
    def backward(self):
        d_dx = 2 * self._input
        return d_dx

In [16]:
class Sequential(Module):
    def __init__(self, modules ,loss):
        super(Sequential,self).__init__()
        
        self._modules = modules
        self._loss = loss
    
    def forward(self,input,target):
        
        y = input.clone()
        for module in self._modules:
            y = module(y)
        
        output = y.clone()
        
        loss = self._loss(y,target)
        
        return loss,output
    
    def backward(self):
        d_dy=self._loss.backward()
        
        for module in reversed(self._modules):
            d_dy = module.backward(d_dy)
            
            
    def zero_grad_(self):
        for module in self._modules:
            module.zero_grad_()

    def update_weights_(self,epsilon):
        for module in self._modules:
            module.update_weights_(epsilon)
        
    
    

In [17]:
hidden1=Linear(2,25)
hidden2=Linear(25,25)
hidden3=Linear(25,25)
output=Linear(25,2)
relu1=ReLU()
relu2=ReLU()
relu3=ReLU()
tanh=Tanh()
loss=LossMSE()

In [18]:
layers=[hidden1,relu1,hidden2,relu2,hidden3,relu3,output,tanh]

In [19]:
network=Sequential(layers,loss)
network(torch.Tensor([0.5,0.5]),torch.Tensor([-1,1]))
network.backward()
network.update_weights_(1e-3)

In [22]:
## hidden1=Linear(2,25)
hidden2=Linear(25,25)
hidden3=Linear(25,25)
output=Linear(25,2)
relu1=ReLU()
relu2=ReLU()
relu3=ReLU()
tanh=Tanh()
loss=LossMSE()

layers=[hidden1,relu1,hidden2,relu2,hidden3,relu3,output,tanh]

network=Sequential(layers,loss)

for i in range(10):
    all_output=[]
    correct=0

    for j,pair in enumerate(zip(train,target_train)):
        train_element,target_element=pair

        loss,output=network(train_element,target_element)
        
#         if 1:
#             print(list(target_element),list(output),loss)
        network.backward()
        network.update_weights_(1e-3)
        network.zero_grad_()
#         print(output)
        right=target_element[1]>target_element[0]
        output_index=output[1]>output[0]
#         print(output)
        all_output.append(output)
        difference=abs(output[1]-output[0])
        output_index=difference<1
        if right==output_index:
            correct+=1

    print('Error rate',(1000-correct)/1000)

Error rate 0.831
Error rate 0.073
Error rate 0.028
Error rate 0.031
Error rate 0.031
Error rate 0.031
Error rate 0.031
Error rate 0.031
Error rate 0.031
Error rate 0.032


# Test

In [23]:
correct=0
for j,pair in enumerate(zip(test,target_test)):

    test_element,target_test_element=pair

    loss,output=network(test_element,target_test_element)
        
        
    right=target_test_element[1]>target_test_element[0]
    difference=abs(output[1]-output[0])
    output_index=difference<1
    if right==output_index:
        correct+=1
    

In [24]:
correct

970